# Projet de Scraping des Réseaux Sociaux

Ce projet consiste à développer des outils de scraping pour extraire des données à partir de différentes plateformes de médias sociaux telles que YouTube, Twitter et Facebook. Les outils développés permettent d'extraire des commentaires, des publications et d'autres informations pertinentes à partir de ces plateformes.

## Installation

Pour utiliser les outils de scraping pour chaque plateforme, suivez les étapes d'installation suivantes :

### Installation des Dépendances

Assurez-vous d'installer les dépendances requises en exécutant la commande suivante :

```bash
pip install pandas twint facebook-scraper google-api-python-client


# TwitterScraper Class

La classe `TwitterScraper` est conçue pour extraire des tweets de Twitter en utilisant l'API Twint. Elle utilise la bibliothèque `twscrape` pour interagir avec l'API Twint de manière asynchrone.

## Méthodes

### `__init__(self)`
- **Description:** Initialise une instance de la classe `TwitterScraper`.
- **Paramètres:** Aucun paramètre requis.

### `gather_tweets(self, query, limit=20)`
- **Description:** Récupère les tweets basés sur une requête donnée.
- **Paramètres:**
  - `query`: La requête à rechercher sur Twitter.
  - `limit`: Le nombre maximum de tweets à récupérer (par défaut: 20).
- **Retourne:** Un DataFrame pandas contenant les données des tweets récupérés.


In [ ]:
import asyncio
from twscrape import API, gather
import pandas as pd

class TwitterScraper:
    def __init__(self):
        self.api = API()

    async def gather_tweets(self, query, limit=20):
        await self.api.pool.add_account("user", "pass", "mail", "mail_pass1") #mail pass optionel  , we can have more than 1
        await self.api.pool.login_all()

        tweets = await gather(self.api.search(query, limit=limit))

        data = []
        for tweet in tweets:
            tweet_data = {
                'ID': tweet.id,
                'Username': tweet.user.username,
                'Content': tweet.rawContent,
                'Date': tweet.date
            }
            data.append(tweet_data)

            print(tweet.id, tweet.user.username, tweet.rawContent)

        df = pd.DataFrame(data)
        return df



# FacebookScraper Class

La classe `FacebookScraper` est conçue pour extraire des commentaires de publications Facebook en utilisant la bibliothèque `facebook_scraper`.

## Méthodes

### `__init__(self)`
- **Description:** Initialise une instance de la classe `FacebookScraper`.
- **Paramètres:** Aucun paramètre requis.

### `getPostData(self, post_url)`
- **Description:** Récupère les données des commentaires d'une publication Facebook spécifique.
- **Paramètres:**
  - `post_url`: L'URL de la publication Facebook.
- **Retourne:** Un DataFrame pandas contenant les données des commentaires extraits.


In [ ]:
import facebook_scraper as fs
import pandas as pd

class FacebookScraper:
    def __init__(self):
        self.MAX_COMMENTS = 100

    def getPostData(self, post_url):
        post_id = post_url.split("/")[-2]
        # Assurez-vous que l'ID ne contient pas d'autres caractères
        post_id=post_id.split("?")[0]
        gen = fs.get_posts(post_urls=[post_id], options={"comments": self.MAX_COMMENTS, "progress": True})
        post = next(gen)
        comments = post['comments_full']
        df = pd.json_normalize(comments, sep='_')
        return df



# YouTubeCommentScraper Class

La classe `YouTubeCommentScraper` est conçue pour extraire des informations sur les vidéos et les commentaires de YouTube en utilisant l'API YouTube Data v3.

## Méthodes

### `__init__(self)`
- **Description:** Initialise une instance de la classe `YouTubeCommentScraper`.
- **Paramètres:**
  - `api_key`: La clé API YouTube pour l'authentification.
  
### `get_video_info(self, video_id)`
- **Description:** Récupère les informations sur une vidéo spécifique.
- **Paramètres:**
  - `video_id`: L'identifiant unique de la vidéo sur YouTube.

### `get_comments_df_for_video(self, video_url, max_results=50)`
- **Description:** Récupère les commentaires d'une vidéo et les stocke dans un DataFrame pandas.
- **Paramètres:**
  - `video_url`: L'URL de la vidéo YouTube.
  - `max_results`: Le nombre maximum de commentaires à récupérer (par défaut: 50).

### `get_comments_info_for_topic(self, topic, max_results=10)`
- **Description:** Récupère les commentaires associés à un sujet donné.
- **Paramètres:**
  - `topic`: Le sujet ou le mot-clé à rechercher sur YouTube.
  - `max_results`: Le nombre maximum de vidéos et de commentaires à récupérer pour ce sujet (par défaut: 10).

### `get_comments_df_for_topic(self, topic, max_results=10)`
- **Description:** Récupère les commentaires associés à un sujet donné et les stocke dans un DataFrame pandas.
- **Paramètres:**
  - `topic`: Le sujet ou le mot-clé à rechercher sur YouTube.
  - `max_results`: Le nombre maximum de vidéos et de commentaires à récupérer pour ce sujet (par défaut: 10).



In [ ]:
import googleapiclient.discovery
import pandas as pd
import re
class YouTubeCommentScraper:
    def __init__(self, ):
       # self.api_key = 'API key'
        self.youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey='ApiKey')

    def get_video_info(self, video_id):
        request = self.youtube.videos().list(part='snippet', id=video_id)
        response = request.execute()

        if 'items' in response:
            video = response['items'][0]
            title = video['snippet']['title']
            description = video['snippet']['description']
            print(f'Title: {title}\nDescription: {description}')

    def get_comments_df_for_video(self, video_url,max_results=50):
        # Extraire l'ID de la vidéo à partir de l'URL
        video_id_match = re.search(r'(?:youtu\.be/|youtube\.com/watch\?v=|youtube\.com/embed/|youtube\.com/v/|youtube\.com/e/|youtube\.com/user/[^/]+/u/|v=)([^"&?/ ]{11})', video_url)
        if video_id_match:
            video_id = video_id_match.group(1)
        else:
            raise ValueError("L'URL de la vidéo YouTube est invalide.")

        comments_request = self.youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                textFormat='plainText',
                maxResults=max_results
            )
        comments_response = comments_request.execute()
        comments_info = []
        for item in comments_response['items']:
                snippet = item['snippet']['topLevelComment']['snippet']
                comments_info.append({
                    'Video Title': snippet.get('videoOwnerChannelInfo', {}).get('videoOwnerChannelTitle', ''),
                    'Comment Date': snippet.get('publishedAt', ''),
                    'Comment': snippet.get('textDisplay', ''),
                    'Likes': snippet.get('likeCount', 0),
                    'Dislikes': snippet.get('dislikeCount', 0)
                })

        return pd.DataFrame(comments_info)


    def get_comments_info_for_topic(self, topic, max_results=10):
        search_request = self.youtube.search().list(
            q=topic,
            part='id',
            type='video',
            maxResults=max_results
        )
        search_response = search_request.execute()

        video_ids = [item['id']['videoId'] for item in search_response['items']]

        comments_info = []
        for video_id in video_ids:
            comments_request = self.youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                textFormat='plainText',
                maxResults=max_results
            )
            comments_response = comments_request.execute()

            for item in comments_response['items']:
                snippet = item['snippet']['topLevelComment']['snippet']
                comments_info.append({
                    'Video Title': snippet.get('videoOwnerChannelInfo', {}).get('videoOwnerChannelTitle', ''),
                    'Comment Date': snippet.get('publishedAt', ''),
                    'Comment': snippet.get('textDisplay', ''),
                    'Likes': snippet.get('likeCount', 0),
                    'Dislikes': snippet.get('dislikeCount', 0)
                })

        return comments_info

    def get_comments_df_for_topic(self, topic, max_results=10):
        comments_info = self.get_comments_info_for_topic(topic, max_results)
        return pd.DataFrame(comments_info)




## Méthodes de Nettoyage des Commentaires

### `clean_youtube_comments(self, df)`
- **Description:** Nettoie les commentaires extraits de YouTube en ne gardant que les commentaires en arabe et en supprimant les hashtags.
- **Paramètres:**
  - `df`: Un DataFrame pandas contenant les données des commentaires de YouTube.
- **Retourne:** Un DataFrame pandas contenant les commentaires nettoyés et leur date correspondante.

### `clean_facebook_comments(self, df)`
- **Description:** Nettoie les commentaires extraits de Facebook en ne gardant que les commentaires en arabe.
- **Paramètres:**
  - `df`: Un DataFrame pandas contenant les données des commentaires de Facebook.
- **Retourne:** Un DataFrame pandas contenant les commentaires nettoyés et leur date correspondante.



In [ ]:
def clean_youtube_comments(self, df):
    cleaned_comments = []
    arabic_letters_pattern = re.compile('[\u0600-\u06FF\s]+')  # Expression régulière pour les lettres en arabe
    french_letters_pattern = re.compile('[a-zA-Z]+')  # Expression régulière pour les lettres en français

    cleaned_data = {'Comment': [], 'Date': []}

    for index, row in df.iterrows():
        # Supprimer les hashtags et ne garder que les lettres en arabe
        comment_text = re.sub(r'#\w+', '', row['Comment'])
        arabic_text = ''.join(arabic_letters_pattern.findall(comment_text))

        # Ajouter le commentaire nettoyé uniquement s'il contient des lettres en arabe et n'est pas vide
        if arabic_text and len(arabic_text.strip()) > 0:
            # Supprimer les commentaires avec plus de 10 lettres en français
            french_text = ''.join(french_letters_pattern.findall(comment_text))
            if len(french_text) <= 10:
                cleaned_data['Comment'].append(arabic_text)
                cleaned_data['Date'].append(row['Comment Date'])

    cleaned_df = pd.DataFrame(cleaned_data)
    return cleaned_df

def clean_facebook_comments(self,df):
    cleaned_data = {'Comment': [], 'Date': []}
    arabic_letters_pattern = re.compile('[\u0600-\u06FF\s]+')  # Expression régulière pour les lettres en arabe
    french_letters_pattern = re.compile('[a-zA-Z]+')  # Expression régulière pour les lettres en français

    for index, row in df.iterrows():
        # Ajouter le commentaire nettoyé uniquement s'il contient des lettres en arabe et n'est pas vide
        comment_text = row['comment_text']
        arabic_text = ''.join(arabic_letters_pattern.findall(comment_text))

        if arabic_text and len(arabic_text.strip()) > 0:
            # Supprimer les commentaires avec plus de 10 lettres en français
            french_text = ''.join(french_letters_pattern.findall(comment_text))
            if len(french_text) <= 10:
                cleaned_data['Comment'].append(arabic_text)
                cleaned_data['Date'].append(row['comment_time'])

    cleaned_df = pd.DataFrame(cleaned_data)
    return cleaned_df

86400

#### Utilisation de YouTubeScraper pour Extraire des Commentaires d'une Vidéo YouTube

```python
from youtube_scraper import YouTubeScraper

scraper = YouTubeScraper()
comments_df = scraper.get_comments_df_for_video("URL_de_la_vidéo")
print(comments_df)



### Exemple d'Utilisation de TwitterScraper

#### Utilisation de TwitterScraper pour Extraire des Tweets basés sur une Requête

```python
from twitter_scraper import TwitterScraper

scraper = TwitterScraper()
tweets_df = await scraper.gather_tweets("python", limit=50)
print(tweets_df)



### Exemple d'Utilisation de FacebookScraper

#### Utilisation de FacebookScraper pour Extraire des Commentaires d'une Publication Facebook

```python
from facebook_scraper import FacebookScraper

scraper = FacebookScraper()
post_url = "URL_de_la_publication"
comments_df = scraper.getPostData(post_url)
print(comments_df)








# Hespress comments scraping
# Outil de Scraping de Sites Web avec Selenium

Cet outil est conçu pour extraire des données à partir de sites web en utilisant Selenium, BeautifulSoup et d'autres bibliothèques Python.

## Description

L'outil utilise Selenium pour automatiser l'interaction avec les sites web, notamment pour charger les pages, cliquer sur des éléments et récupérer le contenu. BeautifulSoup est utilisé pour analyser le HTML des pages web et extraire les données souhaitées.

## Dépendances

Assurez-vous d'installer les dépendances requises en exécutant la commande suivante :

```bash
pip install selenium beautifulsoup4
```


In [ ]:
import os
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd

## Fonction de Scraping des Articles Hespress

Cette fonction permet de rechercher des articles sur le site Hespress en utilisant des mots-clés spécifiés et de récupérer les titres, liens et dates des articles correspondants.

### Paramètres

- `mots` : Les mots-clés à rechercher sur Hespress.
- `rep` (optionnel) : Le nombre de fois à répéter le défilement de la page pour charger plus d'articles (par défaut: 20).


In [ ]:
def hespress_articles(mots,rep=20):
    driver = webdriver.Chrome('C:/Users/sejja/chromedriver')
    link="https://www.hespress.com/?s="+mots
    driver.get(link)

    # determine the rep enough to scroll all the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(rep):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  # Update last_height

    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

    uls = soup.find('div', {'class': 'search_results row'})
    data=[]
    for div in uls.findAll('div', {'class': 'col-12 col-sm-6 col-md-4 col-xl-3'}):  # Iterate over the ResultSet directly
        try:
            title = div.find('h3', {'class': 'card-title'}).text.strip()
            link = div.find('a', {'class': 'stretched-link'}).get('href')
            date = div.find('small', {'class': 'text-muted time'}).text.strip()
            data.append({'Title': title, 'Link': link, 'Date': date})
        except AttributeError:
            pass
    for div in uls.findAll('div', {'class': 'col-12 col-sm-6 col-md-6 col-xl-4 px-2'}):  # Iterate over the ResultSet directly
        try:
            title = div.find('h3', {'class': 'card-title'}).text.strip()
            link = div.find('a', {'class': 'stretched-link'}).get('href')
            date = div.find('small', {'class': 'text-muted time'}).text.strip()
            data.append({'Title': title, 'Link': link, 'Date': date})
        except AttributeError:
            pass
    # Create a DataFrame
    df = pd.DataFrame(data)
    df.shape

    driver.close()
    driver.quit()
    return df


## Fonction d'Extraction de Données depuis une URL

Cette fonction extrait des données à partir d'une URL donnée en utilisant Selenium et BeautifulSoup. Elle récupère le titre de l'article, sa date de publication, les balises associées, ainsi que les commentaires associés à l'article.

### Paramètres

- `url` (str): L'URL de la page web à scraper.

### Retour

- dict: Un dictionnaire contenant les données extraites de la page.
  - `Date` (str): La date de publication de l'article.
  - `Titre` (str): Le titre de l'article.
  - `Tags` (str): Les balises associées à l'article (séparées par des virgules).
  - `Comments` (list): Une liste contenant les commentaires associés à l'article, chaque commentaire étant représenté sous forme de dictionnaire avec les clés suivantes :
    - `comment_date` (str): La date du commentaire.
    - `comment_content` (str): Le contenu du commentaire.
    - `comment_react` (str): La réaction associée au commentaire.


In [ ]:
import os
import time
from selenium import webdriver
from bs4 import BeautifulSoup

def extract_data(url):
    """Extracts data from a given URL using Selenium and BeautifulSoup.

    Args:
        url (str): The URL of the webpage to scrape.

    Returns:
        dict: A dictionary containing the extracted data,
              or None if an error occurs.
    """

    try:
        driver = webdriver.Chrome('C:/Users/sejja/chromedriver')  # Assuming valid path
        driver.get(url)
        time.sleep(3)

        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        titre = soup.find('h1', {'class': 'post-title'})
        if titre:
            titre = titre.get_text().strip()
        else:
            titre = "not available"

        date = soup.find('span', {'class': 'date-post'})
        if date:
            date = date.get_text().strip()
        else:
            date = "not available"

        sections = soup.find('section', {'class': 'box-tags'})
        tags = ""
        if sections:
            for section in sections.findAll('a'):
                if section == sections.findAll('a')[0]:
                    tags = section.get_text().strip()
                else:
                    tags += ","
                    tags += section.get_text().strip()

        comments_area = soup.find('ul', {'class': 'comment-list hide-comments'})
        comments = []
        if comments_area:
            for comment in comments_area.findAll('li', {'class': 'comment even thread-even depth-1 not-reply'}):
                comment_date = comment.find('div', {'class': 'comment-date'})
                comment_content = comment.find('div', {'class': 'comment-text'})
                comment_react = comment.find('span', {'class': 'comment-recat-number'})
                if comment_date and comment_content and comment_react:
                    comments.append({
                        "comment_date": comment_date.get_text(),
                        "comment_content": comment_content.get_text(),
                        "comment_react": comment_react.get_text()
                    })

        return {'Date': date, 'Titre': titre, 'Tags': tags, 'Comments': comments}
    except (NoSuchElementException, TimeoutException) as e:
        print(f"Error encountered while processing {url}: {e}")
        return None
    finally:
        driver.quit()  # Ensure closing the browser even on exceptions



Cet exemple illustre comment utiliser la fonction extract_data pour extraire les données à partir d'une URL spécifiée. Les données extraites sont affichées si aucune erreur ne s'est produite.

In [ ]:
result_df = pd.DataFrame(columns=['Date', 'Titre', 'Tags', 'Comments'])

for d in hespress_articles("رمضان",20)['Link']:
    extracted_data = extract_data(d)
    if extracted_data:
        result_df = result_df.append(extracted_data, ignore_index=True)
result_df

##### Ces exemples illustrent comment utiliser chaque classe de scraping pour extraire des données à partir des plateformes YouTube, Twitter et Facebook. Assurez-vous de remplacer `"URL_de_la_vidéo"`, `"python"` et `"URL_de_la_publication"` par les URL et requêtes appropriées pour votre utilisation spécifique.


### Conclusions des Ressources et Besoins en Développement

1. **Facebook**:
   - Les ressources actuelles permettent le scraping par ID de publication, mais nous souhaitons développer une méthode pour scraper plusieurs publications à partir d'une seule page.
   - Besoin de développer une méthode pour extraire les commentaires de plusieurs publications à partir d'une page Facebook.

2. **YouTube**:
   - Des problèmes surviennent lors du scraping par mot-clé.
   - Besoin de gérer les exceptions pour surmonter les problèmes de scraping sur YouTube.

3. **Twitter**:
   - Besoin de développer davantage la méthode de scraping, y compris l'intégration avec Flask ou un serveur pour gérer les requêtes.
   - Nécessité d'améliorer les fonctionnalités et la robustesse de la méthode de scraping pour Twitter.

4. **Autres Méthodes**:
   - Besoin d'explorer et de développer d'autres méthodes de scraping pour collecter des données de manière efficace et fiable à partir de différentes plateformes de médias sociaux.
   - Recherche de méthodes plus efficaces et fiables pour collecter des données sur les autres plateformes sociales.

En résumé, pour Facebook, nous avons besoin de développer une méthode pour scraper plusieurs publications à partir d'une page. Pour YouTube, il est nécessaire de gérer les exceptions lors du scraping par mot-clé. Pour Twitter, nous devons améliorer la méthode de scraping et envisager une intégration avec Flask ou un serveur. De plus, nous devons explorer d'autres méthodes pour améliorer l'efficacité du scraping sur toutes les plateformes de médias sociaux.